<img style="float: right;" width="30%" src="https://www.robotedu.my/image/robotedu/image/data/all_product_images/product-3866/VK162-Dongle-GPS-Receiver-Navigation-Module-USB-Interface.jpg"/>

# Consolidation exercise - GPS processing

In this exercise you will write code to import and process raw data from a GPS receiver.  In doing so, you will consolidate the following programming skills:
 - reading date from files
 - working with lists
 - writing and using your own functions
 - importing and using functions from built-in libraries

 ## Task - _in pairs_

 You're given the raw GPS log file from the receiver (see image on right) during a short walk in north Bristol.  You need to calculate:
 - the distance traveled
 - the average speed

 > You should work in a pair with someone else in your assigned group.

## Getting started

Either:

- Click [this link](https://colab.research.google.com/github/engmaths/SEMT10002_2024/blob/main/consolidation_exercises/Week_11_gps/week_11_consolidation_gps.ipynb) to open this notebook in Google colab.
- You will need to download the data file using the cell below.

or

- Download it to your local computer using `git clone https://github.com/engmaths/SEMT10002_2024` or just use `git pull` to refresh if you've done this already.
- Navigate to the subfolder `consolidation_exercises/Week_11_gps` and open the notebook `week_11_consolidation_gps.ipynb`.

In [1]:
import sys
if 'google.colab' in sys.modules:
    !wget https://raw.githubusercontent.com/engmaths/SEMT10002_2024/refs/heads/main/consolidation_exercises/Week_11_gps/gps_walk.txt

## What's in the file

The file is in text format and looks like this:

```
$GPRMC,091947.00,A,5128.76976,N,00236.28765,W,0.324,,251124,,,A*6A

$GPVTG,,T,,M,0.324,N,0.600,K,A*20

$GPGGA,091947.00,5128.76976,N,00236.28765,W,1,08,1.16,78.2,M,48.6,M,,*7B

$GPGSA,A,3,30,20,05,13,07,14,11,15,,,,,1.84,1.16,1.43*0F

$GPGSV,3,1,12,05,64,285,33,07,42,059,28,09,08,089,20,11,18,209,42*71
```

These are messages in [NMEA 0183 format](https://tronico.fi/OH6NT/docs/NMEA0183.pdf).  This format originated with marine electronics (think interconnected gizmos for your yacht) but has become the _de facto_ standard for GPS data in all applications.  Plug in the receiver above to any computer and this is the data that comes out of the serial link.  The messages encode all the satellite numbers and signal strengths, but the ones we're most interested in are the GPS fixes, which begin `$GPGGA`.  For example:

```
$GPGGA,091947.00,5128.76976,N,00236.28765,W,1,08,1.16,78.2,M,48.6,M,,*7B
```
where:
 - `091947.00` is the time in UTC (same as UK time since I took the data in winter)
 - `5128.76976,N` means latitude $51^o 28.76976'$ north, 51 degrees and 28.76976 minutes (1 minute is $\frac{1}{60}$ of a degree)
 - `00236.28765,W` means longitude $2^o 36.28765'$ west
 - `1,08` means a regular GPS fix (not differential, which gives higher accuracy by using extra ground stations) from 8 satellites in view
 - `1.16` is the horizontal dilution of precision, a measure of accuracy based on the arrangement of the satellites
 - `78.2,M` means 78.2m above sea level
 - `48.6,M` means 48.6m above GPS' reference ellisoid model of the Earth's surface
 - `*7B` is a checksum

All you're going to need to begin with are the latitude, longitude, and time.


<img style="float: right;" width="30%" src="xymap.png"/>

## Jobs

To accomplish your task of calculating the average speed, you will need to:
- extract just the `$GPGGA` lines in the file
- convert times from text to a usable time value, so you can take the difference between first and last
- convert the latitudes and longitudes from their text formats to decimal numbers
> By convention, east and north are positive, while west and south are negative
- project latitude and longitude coordinates into a grid in metres so you can measure distances
- plot the walk (it should like this image to the right)
- add up total distance and time, then take the average speed. (Hint: search the web for typical walking speed.  It turns out I'm very average.)

Feel free to divide these jobs up between you *in your pair*.  Use functions judiciously to make modular, re-usable code.

> I'd recommend a bit of design work before you start coding.

## Useful Python stuff

### Time handling

Time handling in Python is a bit of a pain as there are different - almost competing - libraries for doing it.  I recommend the `datetime` library, [documented here](https://docs.python.org/3/library/datetime.html).  Here's an example of relevant stuff you can do with it.

In [2]:
from datetime import datetime

time1 = datetime.strptime('11:24:12','%H:%M:%S')
print('time1',time1)
time2 = datetime.strptime('13:48:43','%H:%M:%S')
print('time2',time2)
time_delta = time2 - time1
print('time_delta',time_delta)
delta_seconds = time_delta.total_seconds()
print('delta_seconds',delta_seconds)

time1 1900-01-01 11:24:12
time2 1900-01-01 13:48:43
time_delta 2:24:31
delta_seconds 8671.0


### Coordinate projections

There's a lovely big rabbit hole you can go down on the subject of mapping projection and coordinate transforms.  Happily, the excellent [`pyproj` library](https://pyproj4.github.io/pyproj/stable/index.html) makes all this easily accessible through its [`Transformer` object](https://pyproj4.github.io/pyproj/stable/index.html).

Coordinate systems are catalogued using EPSG codes, including two relevant for this task:
 - EPSG:4326 is the GPS coordinate system, using latitude and longitude angles on an ellisoidal approximation of the Earth's surface
 - EPSG:27700 is the UK Ordnance Survey national grid coordinate system, in metres east and north from a point somewhere off the south west coast

There is a nice example of what you need to do [here](https://pyproj4.github.io/pyproj/stable/examples.html#step-2-create-transformer-to-convert-from-crs-to-crs).  You can check locations using different coordinate systems on [this free website](https://gridreferencefinder.com/) (with apologies for adverts).

You might need to install the pyproj library using:
- `%pip install pyproj` on Colab
- `python -m pip install pyproj` on your local PC.


> And don't forget that the square root function lives in the `math` library

## Submitting

Both members of each pair should submit an identical `.py` file to the *Consolidation Exercise 2* submission point on Blackboard.

> Please state both your names in a comment at the top of your file.